# Фильтры калмана на примерах

## Одномерный случай

### Постановка задачи

Допустим у нас есть человек, стоящий на месте и желающий определить свое место положения. В текущий момент времени ему доступны два источника GPS и ГЛАНАС. Эти источники не точны и присылают координату с некоторой погрешностью n1 для GPS и n2 для ГЛАНАС соответственно. Из технической документации нам известно, что источник GPS в среднем имеет ошику $\sigma_1$, а источник ГЛАНАС $\sigma_2$. Причём эти ошибки имеют нормальное распределение с нулевым математическим ожиданием, то есть они в среднем дают правильный ответ. Задача: используя эти два наблюдения найти более точную оценку координаты X.

Итак, опишем задачу более формально. Координату, полученную от GPS обозначим как $z_1$, она представляет собой истинное значение координаты $x$ плюс некоторая случайная величина (шум) $n_1$ с диспепсией $\sigma_1^2$ и нулевым математическим ожиданием

$z_1=x+n_1$ $n_1\sim N(0,\sigma_1^2)$

Аналогично для второго есть источника

$z_2=x+n_2$ $n_2\sim N(0,\sigma_2^2)$

![two_sourcre](images/kalman_filter_algorithm/image001.png)

Задача: построить алгоритм объединяющий эти два наблюдения и дающий более точную оценку координаты X

### Основная идея

Давай  порассуждаем. Мы не знаем точного значения координаты Х, он точно знаем что она, с одной стороны, лежит в диапазоне $[z_1-\sigma_1, z_1+\sigma_1]$, с другой стороны ее диапазон равен $[z_2-\sigma_2,z_2+\sigma_2]$. Причём, наиболее вероятным значением X, с точки зрения первого источника является $z_1$, а с точки зрения второго источника $z_2$. Хотелось бы построить алгоритм, принимающий два этих диапазона и учитывающий, что наиболее вероятными значениями является или $z_1$, или $z_2$. Как построить такой алгоритм ?

Мы знаем что ошибка у источников GPS и ГЛАНАС имеет нормально распределение с нулевым математическим ожиданием, а наблюдения $z_1$ и $z_2$ являются, по сути, суммой этой ошибки некоторой константы Х. 

$z_1=x+n_1$ $n_1\sim N(0,\sigma_1^2)$

$z_2=x+n_2$ $n_2\sim N(0,\sigma_2^2)$

Следовательно, величины $z_1$ и $z_2$ также нормально распредельный, но их средним значение является искомое Х. Oбозначим их распределения как $Z_1$ и $Z_2$

$Z_1 \sim N(X,\sigma_1^2)$, $Z_2\sim N(X,\sigma_2^2)$

Искомое Х нам неизвестно. Однако известно, что наиболее вероятному значению в нормальном распределение соответствует среднее значение и что, с точки зрения первого источника, наиболее вероятным значением является $z_1$, a с точки зрения второго $z_2$. Следовательно, лучшим предположением для распределение $Z_1$ и $Z_2$ на текущем шаге является.

$Z_1 \sim N(z_1,\sigma_1^2)$, $Z_2\sim N(z_2,\sigma_2^2)$

В этих распределенияx есть всё что нам нужно. С одной стороны они учитывают, что наиболее вероятным значением является $z_1$ илм  $z_2$, а с другой стороны, они учитывают диапазон возможных значение, который задаётся их дисперсиями. Итак, у нас есть две вероятности. Как найти вероятность того, что они обе истины? Для этого их нужно просто перемножить.

$$N(z_1,\sigma_1^2) * N(z_2,\sigma_2^2) = N(z_3,\sigma_3^2)$$

Это и есть фильтр Калмана. Нам осталось вывести рекуррентные формулы для новый дисперсии и среднего.

В <a href="https://ccrma.stanford.edu/~jos/sasp/Product_Two_Gaussian_PDFs.html">интернете</a> можно найти формулу следующего вида

$$ z_3 = \frac{z_1 \sigma_2^2 + z_2 \sigma_1^2}{\sigma_2^2+\sigma_1^2} $$

$$ \sigma_3 = \frac{\sigma_1^2 \sigma_2^2}{\sigma_2^2+\sigma_1^2}  $$

Приведём их рекуррентному виду

$$ \sigma_3 = \frac{\sigma_1^2 \sigma_2^2}{\sigma_2^2+\sigma_1^2}=\frac{\sigma_1^4+\sigma_1^2 \sigma_2^2-\sigma_1^4}{\sigma_2^2+\sigma_1^2}=\frac{\sigma_1^2(\sigma_2^2+\sigma_1^2)-\sigma_1^4}{\sigma_2^2+\sigma_1^2}=\sigma_1^2-\frac{\sigma_1^4}{\sigma_2^2+\sigma_1^2}$$

$$z_3 = \frac{z_1 \sigma_2^2 + z_2 \sigma_1^2}{\sigma_2^2+\sigma_1^2}=\frac{\sigma_2^2}{\sigma_2^2+\sigma_1^2}*z_1+\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}*z_2=\frac{\sigma_2^2}{\sigma_2^2+\sigma_1^2}*z_1+\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}*z_1+\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}*z_2-\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}*z_1=  $$

$$=\frac{\sigma_2^2+\sigma_1^1}{\sigma_2^2+\sigma_1^2}*z_1+\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}*(z_2-z_1)=z_1+\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}*(z_2-z_1)$$

итоговые формулы

$$ \sigma_3 =\sigma_1^2-\frac{\sigma_1^4}{\sigma_2^2+\sigma_1^2}$$

$$z_3 = z_1+\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}*(z_2-z_1)$$

Два этих уравнения имеют общий множитель. Запишем его отдельно и перепишем формулы

$$K=\frac{\sigma_1^2}{\sigma_2^2+\sigma_1^2}$$

$$ \sigma_3 =\sigma_1^2-K\sigma_1^2$$

$$z_3 = z_1+K*(z_2-z_1)$$

Kоэффициент K называют называют коэффициентом усиления Калмана. Он варьируется от нуля до единицы и представляет собой долю новый информация,  которая возьмётся из нового наблюдения предыдущего значения.

Очевидно что, источников присылающих текущую координату, может быть не два а множество. Тогда наши формулы просто примут рекурсивный вид

$$K_{t+1}=\frac{\sigma_{t-1}^2}{\sigma_{t-1}^2+\sigma_t^2}$$

$$ \sigma_{t+1} =\sigma_{t-1}^2-K_{t+1}\sigma_{t-1}^2$$

$$z_{t+1} = z_{t-1}+K_{t+1}*(z_t-z_{t-1})$$

Каждый раз, получая значение текущей координаты от нового источника, мы будем получать более точный результат.

## Многомерный случай

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import numpy as np
import random

### Постановка задачи

Допустим у нас есть робот с датчиком на голове. Датчик возвращает кординаты Х и У через равные промежутки времени. Показания дата не точны и имеют дисперсию ss. Робот движется с постоянной не известной нам скоростью. На его координаты также влияют внешние факторы, его колеса  могут проскальзывать или застревать в грязи. Смоделируем это как шум с дисперсией ms. Задача найти скоростью с помощью фильтра Калмана. 

### Класс робот

In [2]:
class Robot:
    def __init__(self, ss, ms):
        self.ss = ss
        self.ms = ms
        self.vx = random.randint(1, 10)
        self.vy = random.randint(1, 10)
        self.x = random.randint(0, 10)
        self.y = random.randint(0, 10)

    def sn(self):
        return np.random.normal(0, self.ms, 1)[0]

        
    def move(self):
        self.x = self.vx + self.x + self.sn()
        self.y = self.vy + self.y + self.sn()

    

In [3]:
r = Robot(0.5,0.3); r.x; 

In [4]:
r.x

0

In [5]:
r.vx

10

In [6]:
r.move()

In [7]:
r.x

10.037815964566308